In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from google.cloud import bigquery
import dill
from tqdm import tqdm_notebook
from datetime import datetime
from datetime import date
from datetime import timedelta
import warnings
import pickle
warnings.filterwarnings('ignore')


client = bigquery.Client.from_service_account_json(r'xxx.json')
print('Укажи начало периода окна в формате "20190101"')
start_period = str(input())
print('Укажи конец периода окна в формате "20191201"')
end_period = str(input())
start_periodmin1 = (datetime.strptime(start_period, '%Y%m%d') - timedelta(1)).strftime('%Y%m%d')
q_statga_end_start = ("SELECT DISTINCT * FROM (SELECT date, fullVisitorId, visitId, TIMESTAMP_SECONDS(visitStartTime) as starttime_UTC, hits.page.pagePath, trafficSource.source, trafficSource.medium, trafficSource.campaign, COALESCE(hits.eventInfo.eventCategory,'PageView') as ev_type, hits.eventInfo.eventAction, clientID, device.deviceCategory FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND date != '20190909' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND ((hits.eventInfo.eventCategory = 'clients' AND hits.eventInfo.eventAction = 'nk') OR (hits.eventInfo.eventCategory = 'leads' AND hits.eventInfo.eventAction LIKE 'new_contact%') OR hits.eventInfo.eventCategory = 'request_send_all') AND clientid NOT IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '{}' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20150101' AND '{}')))) ORDER BY fullVisitorId, starttime_UTC ASC)").format(start_period,end_period,start_period,end_period,start_period,end_period,start_period,end_period,start_periodmin1)
#q_statga = ("SELECT DISTINCT * FROM (SELECT date, fullVisitorId, visitId, TIMESTAMP_SECONDS(visitStartTime) as starttime_UTC, hits.page.pagePath, trafficSource.source, trafficSource.medium, trafficSource.campaign, COALESCE(hits.eventInfo.eventCategory,'PageView') as ev_type, hits.eventInfo.eventAction, clientID, device.deviceCategory FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND date != '20190909' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND ((hits.eventInfo.eventCategory = 'clients' AND hits.eventInfo.eventAction = 'nk') OR (hits.eventInfo.eventCategory = 'leads' AND hits.eventInfo.eventAction LIKE 'new_contact%') OR hits.eventInfo.eventCategory = 'request_send_all') AND clientid NOT IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20150101' AND '{}')))) ORDER BY fullVisitorId, starttime_UTC ASC)").format(start_period,start_period,start_period,start_period,start_periodmin1)
#q_statga = ("SELECT DISTINCT * FROM (SELECT date, fullVisitorId, visitId, TIMESTAMP_SECONDS(visitStartTime) as starttime_UTC, hits.page.pagePath, trafficSource.source, trafficSource.medium, trafficSource.campaign, COALESCE(hits.eventInfo.eventCategory,'PageView') as ev_type, hits.eventInfo.eventAction, clientID, device.deviceCategory FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '20190601' AND date != '20190909' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '20190601' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '20190601' AND ((hits.eventInfo.eventCategory = 'clients' AND hits.eventInfo.eventAction = 'nk') OR (hits.eventInfo.eventCategory = 'leads' AND hits.eventInfo.eventAction LIKE 'new_contact%') OR hits.eventInfo.eventCategory = 'request_send_all') AND clientid NOT IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '{}' AND '20190601' AND clientid IN (SELECT DISTINCT(clientid) FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20150101' AND '{}')))) ORDER BY fullVisitorId, starttime_UTC ASC)").format(start_period,start_period,start_period,start_period,start_periodmin1)

q_j_statga = client.query(q_statga_end_start)
statga = q_j_statga.to_dataframe()
statga.fillna(0,inplace=True)
statga['sourcemedium'] = statga['source'].astype(str) + statga['medium'].astype(str)
statga.sort_values(['clientID','starttime_UTC'], ascending=[False,True], inplace=True)

#statga[(statga['clientID'].str.contains('315691920.1550135471',na=False))]

In [ ]:
connection_DB_CRM = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}',server = 'BD-SRV-DWH' , database = 'OpenDWH')
sql_crm="SELECT * FROM [datamartMarketing].[vwCrmProspect] WHERE date_creation >= '2019-01-01'"
SQL_CRM_all = pd.read_sql(sql_crm,connection_DB_CRM)

coub_old = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Coub1_2016_2018.xlsx').fillna(0)
coub_new = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Coub1_2019.xlsx').fillna(0)
coub_new['Месяц'] = coub_new['Месяц'].apply(lambda x: str(x).replace(' 2019',''))
coub_total = pd.concat([coub_old, coub_new], axis=0)


In [ ]:
nk = set(coub_total[(coub_total['К-во персон 5 - 50 т.р.'] == 1)&(coub_total['Год'] == 2019)]['GUID'])
coub_tot_ui = set(SQL_CRM_all[SQL_CRM_all['GUID'].isin(nk)]['cookie_id'])

In [ ]:
len(nk)  #число уник гуид нк в кубе

In [ ]:
SQL_CRM_all[SQL_CRM_all['GUID'].isin(nk)]['GUID'].nunique() #число уник гуид нк в crm

In [ ]:
len(coub_tot_ui) #число смэченных уник клидов нк в crm

In [ ]:
stat_gr = statga.groupby('clientID').agg({'sourcemedium': list, 'ev_type': list}).reset_index()
stat_gr['ev_type_str'] = stat_gr['ev_type'].astype(str)
stat_gr = stat_gr[stat_gr['ev_type_str'].str.contains('request_send_all')].drop('ev_type_str', axis=1)
stat_gr.shape

In [ ]:
stat_gr_nk = stat_gr[stat_gr['clientID'].isin(coub_tot_ui)]
stat_gr_nk.shape

In [ ]:
stat_gr_nk['index'] = stat_gr_nk['ev_type'].apply(lambda x: x.index("request_send_all"))

In [ ]:
from itertools import groupby
indexes = list(stat_gr_nk['index'])

list_chains = []
for i,e in enumerate(list(stat_gr_nk['sourcemedium'])):
    l1 = [m[0] for m in groupby(e[0:indexes[i]+1])]
    list_chains.append(l1)

In [ ]:
from statistics import mean 
from collections import Counter
import numpy as np
from numpy import median

dict_zayvki = dict.fromkeys(list(set(np.concatenate(list_chains,axis=0))), [])
for key in dict_zayvki:
    dict_zayvki[key] = []
    
dict_firstzayvka = dict.fromkeys(list(set(np.concatenate(list_chains,axis=0))), [])
for key in dict_firstzayvka:
    dict_firstzayvka[key] = []
    
dict_touches = dict.fromkeys(list(set(np.concatenate(list_chains,axis=0))), [])
for key in dict_touches:
    dict_touches[key] = []

dict_first = dict.fromkeys(list(set(np.concatenate(list_chains,axis=0))), [])
for key in dict_first:
    dict_first[key] = []
    
dict_lastnondirect = dict.fromkeys(list(set(np.concatenate(list_chains,axis=0))), [])
for key in dict_lastnondirect:
    dict_lastnondirect[key] = []

In [ ]:
fg = ['yandex','google','direct']
for i1, k in enumerate(fg[::-1]):
    if k.count('irec') == 0:
        break
fg = fg[::-1][i1:][::-1]
fg[-1]

In [ ]:
for e in list_chains:
    for i1, k in enumerate(e[::-1]):
        if k.count('irec') == 0:
            break
    e = e[::-1][i1:][::-1]
    
    dict_zayvki[e[-1]].append(1)
    dict_firstzayvka[e[0]].append(1)
    
    for k in e:
        dict_touches[k].append(1)

    e_fc = e
    df_fc = 1/sum([i/np.exp(i) for i in range(1,len(e_fc)+1)])
    values_l_fc = [(i/np.exp(i))*df_fc for i in range(1,len(e_fc)+1)]
    for index,m in enumerate(e_fc):
        dict_first[m].append(values_l_fc[index])
    
    e_lsc = e[::-1]
    df_lsc = 1/sum([i/np.exp(i) for i in range(1,len(e_lsc)+1)])
    values_l_lsc = [(i/np.exp(i))*df_lsc for i in range(1,len(e_lsc)+1)]
    for index,m in enumerate(e_lsc):
        dict_lastnondirect[m].append(values_l_lsc[index])


In [ ]:
e = ['yandex','google','direct']
e_lsc = e[::-1]
df_lsc = 1/sum([i/np.exp(i) for i in range(1,len(e_lsc)+1)])
values_l_lsc = [(i/np.exp(i))*df_lsc for i in range(1,len(e_lsc)+1)]
values_l_lsc

In [ ]:
df_z = pd.Series(dict_zayvki, index=dict_zayvki.keys()).reset_index().rename(columns={ 0: 'Zayavki'})
df_fz = pd.Series(dict_firstzayvka, index=dict_firstzayvka.keys()).reset_index().rename(columns={ 0: 'First_Zayvka'})
df_touches = pd.Series(dict_touches, index=dict_touches.keys()).reset_index().rename(columns={ 0: 'Touches'})
df_fc = pd.Series(dict_first, index=dict_first.keys()).reset_index().rename(columns={ 0: 'FC'})
df_lcs = pd.Series(dict_lastnondirect, index=dict_lastnondirect.keys()).reset_index().rename(columns={ 0: 'LFC'})

In [ ]:
df_all = df_z.merge(df_fc,on='index',how='left').merge(df_lcs,on='index',how='left').merge(df_touches,on='index',how='left').merge(df_fz,on='index',how='left')

In [ ]:
df_all['Touches_Total'] = df_all['Touches'].apply(lambda x: sum(x))
df_all['Zayavki_Total'] = df_all['Zayavki'].apply(lambda x: sum(x))
df_all['First_Zayavki_Total'] = df_all['First_Zayvka'].apply(lambda x: sum(x))
df_all['FC_Total'] = df_all['FC'].apply(lambda x: median(x))
df_all['LFC_Total'] = df_all['LFC'].apply(lambda x: median(x))

In [ ]:
df_Old_SourceMedium = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Sprаvochnik_Old_SourceMedium.xlsx')
df_New_SourceMedium = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Sprаvochnik_New_SourceMedium.xlsx')
df_spravochnik_all_campaigns = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Sprаvochnik_All_Campaigns.xlsx')
df_group_campaigns = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Sprаvochnik_Group_by_Campaigns.xlsx')

df_all = df_all.rename(columns={'index': 'SourceMedium'}).merge(df_Old_SourceMedium,on='SourceMedium',how='left').rename(columns={'SourceMedium_Type': 'Источник по старому'}).merge(df_New_SourceMedium,on='Источник по старому',how='left')

In [ ]:
df_all.groupby('Источник по новому').agg({'Touches_Total':'sum','Zayavki_Total':'sum','First_Zayavki_Total':'sum','FC_Total':'median','LFC_Total':'median'}).to_excel('Ассоциированные конверсии.xlsx')